In [5]:
with open('reviews.txt', encoding='utf-8') as file:
    raw_reviews = file.readlines()

In [27]:
with open('labels.txt', encoding='utf-8') as file:
    raw_labels = file.readlines()

In [6]:
tokens = list(map(str.split, raw_reviews)) # статьи

In [24]:
vocab = list({word for token in tokens for word in token if len(word) > 0}) # словарь всех слов

In [25]:
word2index = {word: idx for idx, word in enumerate(vocab)} #  словарь индексов слов

In [29]:
input_dataset = [[word2index[word] for word in token] for token in tokens]

In [33]:
target_dataset = [1 if label == 'positive\n' else 0 for label in raw_labels]

In [35]:
import sys
import numpy as np
np.random.seed(1)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

alpha, iterations = (0.01, 2)
hidden_size = 100

weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1

correct,total = (0,0)
for iter in range(iterations):
    
    # train on first 24,000
    for i in range(len(input_dataset)-1000):

        x,y = (input_dataset[i],target_dataset[i])
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0)) #embed + sigmoid
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2)) # linear + softmax

        layer_2_delta = layer_2 - y # compare pred with truth
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) #backprop

        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha

        if(np.abs(layer_2_delta) < 0.5):
            correct += 1
        total += 1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter:'+str(iter)\
                             +' Progress:'+progress[2:4]\
                             +'.'+progress[4:6]\
                             +'% Training Accuracy:'\
                             + str(correct/float(total)) + '%')
    print()
correct,total = (0,0)
for i in range(len(input_dataset)-1000,len(input_dataset)):

    x = input_dataset[i]
    y = target_dataset[i]

    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    
    if(np.abs(layer_2 - y) < 0.5):
        correct += 1
    total += 1
print("Test Accuracy:" + str(correct / float(total)))

Iter:0 Progress:95.99% Training Accuracy:0.8183333333333334%%
Iter:1 Progress:95.99% Training Accuracy:0.8487291666666666%
Test Accuracy:0.853


In [38]:
883*2095*1200*2/(2400*2400*13300)

0.05795379072681704